# Entrega Análisis y Regresión Lineal - 2025

## Integrantes:

- Defelipe, Bianca Eugenia - Legajo: 21341/7.
- Gonzalez, Joaquín Manuel - Legajo: 21247/1.
- Loyola, Yanella Nicole - Legajo: 20912/8.

## Dataset Elegido

El Dataset *Obesity Levels Based On Eating Habits and Physical Activites* fue elegido por contener los suficientes atributos **númericos continuos** para el adecuado **análisis estadístico y matemático** que es requerido para satisfacer los objetivos de la **entrega asignada**.

### Sobre los atributos del Dataset

El dataset presenta una combinación equilibrada de **variables continuas, binarias y categóricas**, que permiten realizar un análisis de los factores asociados al nivel de obesidad. Entre sus atributos destacan **variables continuas** como la **edad, altura, peso, cantidad de agua consumida y frecuencia de actividad física**, las cuales son adecuadas para la aplicación de métodos de regresión lineal.

### Motivación Personal 

La **motivación principal** que nos hizo elegir este dataset como grupo radica en que la obesidad es actualmente uno de los principales problemas de salud pública a nivel mundial, con consecuencias tanto físicas como psicológicas.

## Regresión Lineal Simple

### Definir la variable respuesta y las variables predictoras, justificando el motivo de la elección de estas

**Variable Respuesta**
- *Weight (Peso):* Variable continua que refleja directamente el estado corporal del individuo y constituye uno de los principales indicadores utilizados para determinar niveles de sobrepeso u obesidad.

**Variables Predictoras**
- *Height (Altura):* Variable continua que tiene una relación fisiológica con el peso ya que el tamaño corporal condiciona la masa total del individuo. En términos estadísticos, se espera una relación lineal positiva: a mayor altura, mayor peso promedio.
- *Age (Edad):* Variable continua que refleja como el metabolismo y la composición corporal varían con la edad. En general, el peso tiende a aumentar con los años debido a cambios hormonales y reducción de la actividad física, lo que la convierte en una variable predictora de interés.
- *CH20 (Consumo de Agua Diaria):* Variable continua que refleja hábitos saludables y de hidratación. Una ingesta adecuada de agua se asocia a una mejor regulación metabólica y puede influir indirectamente en el control del peso corporal.
- *FAF (Frecuencia de Actividad Física):* Variable continua que representa el nivel de actividad física del individuo. La práctica regular de ejercicio contribuye a un mayor consumo calórico y a la regulación del metabolismo lo que suele asociarse con un mejor control del peso corporal.
- *TUE (Tiempo de Uso de Dispositivos Tecnológicos):* Variable entera que actúa como un indicador de sedentarismo. El uso prolongado de dispositivos tecnológicos se vincula con menor gasto energético y mayor probabilidad de aumento de peso.
- *NCP (Número de Comidas Diarias):* Variable continua que representa la cantidad de ingestas completas que la persona realiza en un día. Mantener una frecuencia adecuada de comidas contribuye a una mejor distribución calórica y puede influir en el mantenimiento del peso corporal.
